# Logistic Regression

In this exercise, we will implement a logistic regression for binary classification.

In [1]:
import numpy as np
import pandas as pd

## Model Function

Given a multiset of examples $D = \{(\mathbf{x}_1, c_1), ..., (\mathbf{x}_n, c_n)\}$ (the dataset), our learning task is to predict a class output under a logistic model function

$$ y(x) = \sigma(\mathbf{w}^T\mathbf{x}),$$

where $\sigma$ is the sigmoid function

$$\sigma(z) = \frac{1}{1 + e^{-z}}.$$

The sigmoid function maps the space of real numbers to the $(0,1)$ interval, which allows us to interpret the output of the model function as probability for the event $\mathbf{C} = 1$.

**Exercise**: implement the sigmoid function and the model function.


In [2]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [3]:
def model(w, X):
    return sigmoid(X.dot(w.T))

## Loss

The loss function quantifies the error made by the model. Importantly, the loss has to be derivable, so we cannot simply use accuracy here. Instead, we rely on the pointwise logistic loss:

$$
l_{\sigma}(c, y(\mathbf{x})) = \begin{cases} -\log(y(\mathbf{x})) & \text{ if } c = 1 \\  -\log(1 - y(\mathbf{x})) & \text{ if } c = 0 \end{cases}.
$$

Please refer to the lecture slides for the derivation of this loss function.

**Exercise**: implement the pointwise logistic loss for binary classification.

In [4]:
def loss(y, c):
    """
    Pointwise logistic loss

    :param y: prediction
    :param c: ground-truth label
    :return: pointwise logistic loss
    """
    return c * -np.log(y) + (1 - c) * -np.log(1 - y)

## Convergence

The last important part needed to assemble the training loop is the convergence function. As stated in the lecture slides, the function `convergence` analyses the norm of the loss gradient,

$$||\nabla L_\sigma(\mathbf{x}_t)|| = ||\sum_{(\mathbf{x},c)\in D}(c-y_t(\mathbf{x}))\cdot\mathbf{x}||,$$

and compares it to a small positive bound $\epsilon$. In addition, it may check if an upper bound for the number of iterations is reached. In comparison to the lecture, we simplify the implementation by calculating the pointwise loss gradient for each example individually, and then use the inner training loop to sum over all examples.

This means that the convergence function for a single example will calculate $(c-y_t(\mathbf{x}))\cdot\mathbf{x}$ only.



**Exercise**: implement the convergence function for a single example.


In [5]:
def convergence(x, c, y):
    """
    Convergence function for a single example.

    :param x: the feature vector for the example
    :param c: the ground-truth class of the example
    :param y: the model prediction for the example
    :return: the pointwise direction of the steepest loss descent
    """
    return (c - y) * x

## Batch Gradient Descent

In order to fit our model, we rely on the logistic Batch Gradient Descent (BGD$_{\sigma}$) algorithm.

**Exercise**: implement the BGD$_{\sigma}$ algorithm from the lecture. Some code is already filled in.

*Remarks*:
   - calculate the global logistic loss at every iteration and save the sum of loss values in an array; we want to visualize the loss after training, so it should be returned together with the optimal weights.
   - calculate the convergence for each example using the function defined before and sum up their individual contributions; decide after the inner training loop whether convergence is reached or not. Remember to calculate the norm of the summed convergence values!

In [6]:
def bgd(D, eta: float, eps: float, max_iter: int = 1000):
    """
    Batch Gradient Descent

    :param D: multiset of examples (x, c), where x is a training vector and c is a class in {0, 1}.
    :param eta: learning rate, a small positive constant
    :param eps: epsilon value for convergence
    :param max_iter: maximum number of training iterations (epochs)
    :return: w (weight vector), l (loss vector)
    """
    # Add bias dimension to data, inserting an extra 1 to every feature array
    D = [(np.hstack([1, x]), c) for (x, c) in D]
    # Initialize random w vector with random values + bias
    w = (np.random.random(size=D[0][0].shape))
    # Initialize an array to keep track of the loss
    l = np.zeros(shape=max_iter, dtype=np.float64)

    # Training loop
    t = 0
    while True:
        # Reset w_delta
        w_delta = 0
        # Reset conv
        conv = np.zeros_like(w)
        # Batch loop
        for (x, c) in D:
            # Call model function to calculate prediction
            y = model(w, x)
            # Calculate the loss and increment l[t] by it
            l[t] += loss(y, c)
            # Calculate the convergence and increment conv by it
            conv += convergence(x, c, y)
            # Calculate delta
            delta = c - y
            # Add the step to w_delta
            w_delta += eta * delta * x
        # Update weights
        w += w_delta
        # Update timestep
        t += 1
        # Check if convergence is reached; remember to calculate the vector norm!
        if np.linalg.norm(conv) <= eps or t >= max_iter:
            break
    # Return final weights and loss history (capped to current time step)
    return w, l[:t]

## Predictions

Using BGD, we can infer optimal weights for our model function. Yet, the output of the model function is the *probability* for each class, not the actual class label. Therefore, we need a `predict` function that takes a (batch of) training example(s), and returns the most probable class label. It is defined as follows:

$$
\text{predict}(\mathbf{x}) = \begin{cases}1 & \text{ if } y(\mathbf{x}) >= 0.5 \\ 0 & \text{ if } y(\mathbf{x}) < 0.5\end{cases}

$$

**Exercise**: implement a `predict` function.

*Remarks*: remeber to also insert the bias value to every feature vector here!

In [7]:
def predict(X, w):
    """
    Predict class labels for given data.

    :param X: data vector
    :param w: weight vector
    :return: predicted 0/1 class labels
    """
    X = np.hstack([np.ones(shape=(X.shape[0], 1)), X])
    return np.where(model(w, X) >= 0.5, 1, 0)

## Example Classification Task

To test the capabilities of the model, we conduct a sample classification task. The goal is to classify different wines (described by different features such as *Alcohol*, *Malic Acid*, *Magnesium*, etc.) into categories. Below, the dataset is loaded and the associated description with more info on its content is printed.

In [8]:
from sklearn.datasets import load_wine
data = load_wine()
print(data.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            10.6  30.0    19.5   3.3
    Ma

In [9]:
# Load the feature vectors X
X = data.data
# Load the class labels C
C = data.target

Initially, wines are divided into three classes; to achieve a binary setting, we simply collapse label 2 and 3.

**Exercise**: modify the class labels such that class 2 and 3 (labels 1 and 2) are both represented by label 1.

In [10]:
C = np.where(C == 0, 0, 1)

Investigating the value ranges of features, we can observe that they are distributed extremely different: while some features are in the 0-1 range, feature 13 goes up to 1680.

In [11]:
pd.DataFrame(np.array([X.min(axis=0), X.max(axis=0)]).T, columns=["Minimum", "Maximum"])

,Minimum,Maximum
0,11.03,14.83
1,0.74,5.80
2,1.36,3.23
3,10.60,30.00
4,70.00,162.00
5,0.98,3.88
6,0.34,5.08
7,0.13,0.66
8,0.41,3.58
9,1.28,13.00


This warrants *feature scaling*.  Intuitively, if we have incompatible ranges, features with larger ranges will dominate. Therefore, we apply *z-Standardization*, which means that every feature is rescaled to have a distribution of mean 0 and standard deviation of 1. We can use the existing implementation from `sklearn`.

In [12]:
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit_transform(X)

Now, all features have compatible ranges:

In [13]:
pd.DataFrame(np.array([X.min(axis=0), X.max(axis=0)]).T, columns=["Minimum", "Maximum"])

,Minimum,Maximum
0,-2.434235,2.259772
1,-1.432983,3.109192
2,-3.679162,3.156325
3,-2.671018,3.154511
4,-2.088255,4.371372
5,-2.107246,2.539515
6,-1.695971,3.062832
7,-1.868234,2.402403
8,-2.069034,3.485073
9,-1.634288,3.435432


**Exercise**: assemble $D$ and fit the model using $BGD_{\sigma}$.

*Remarks*:
- use 0.0001 as learning rate
- use 40 as epsilon
- use 1000 max iterations

In [14]:
D = list(zip(X, C))
w, l = bgd(D, 1e-4, 40, max_iter=1_000)

**Exercise**: predict classes for every sample in $C$. Evaluate your result using either the `sklearn` evaluation metrics, or your own implementation from the last exercise. Play around with the model parameters and see if you can improve the results.

In [15]:
from sklearn.metrics import classification_report

print(classification_report(y_true=C, y_pred=predict(X, w)))

              precision    recall  f1-score   support

           0       0.83      0.93      0.88        59
           1       0.96      0.91      0.94       119

    accuracy                           0.92       178
   macro avg       0.90      0.92      0.91       178
weighted avg       0.92      0.92      0.92       178

